In [1]:
import pyspark.sql.functions as f

In [2]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("CleaningData")\
    .config('spark.driver.extraClassPath','/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.6.0.jar')\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/11/02 10:37:16 WARN Utils: Your hostname, LAPTOP-GRLQJBNM resolves to a loopback address: 127.0.1.1; using 172.27.87.196 instead (on interface eth0)
23/11/02 10:37:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 10:37:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

user = os.getenv("user")
password = os.getenv("password")

In [4]:
jdbc_url = "jdbc:postgresql://localhost:5432/air"

connection_properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

uncleaned_data = spark.read.jdbc(url=jdbc_url, table="test_table_create_testing_final", properties=connection_properties)

In [5]:
uncleaned_data.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: date (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [6]:
uncleaned_data.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|  2018-01-07|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  2018-01-15|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  2018-08-01|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AN

In [7]:
uncleaned_data = uncleaned_data.withColumn("Rating", f.when(f.col("Rating") == "NaN", "0.0").otherwise(f.col("Rating")))

In [8]:
cleaning_data = uncleaned_data.withColumn("Current Ver", f.when(f.col("Current Ver") == "Varies with device", None).otherwise(f.col("Current Ver")))

In [9]:
cleaning_data.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|  2018-01-07|      1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  2018-01-15|      2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  2018-08-01|      1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|F

In [10]:
cleaning_data = cleaning_data.withColumn("Device_Current_Version_Dependency", f.when(f.col("Current Ver").isNull(), True).otherwise(False))

In [11]:
cleaning_data = cleaning_data.withColumn("Size", f.when(f.col("Size") == "Varies with device", None).otherwise(f.col("Size")))

In [12]:
cleaning_data = cleaning_data.withColumn("Device_Size_Dependency", f.when(f.col("Size").isNull(), True).otherwise(False))

In [13]:
cleaning_data = cleaning_data.withColumn("Android Ver", f.when(f.col("Android Ver") == "Varies with device", None).otherwise(f.col("Android Ver")))

In [14]:
cleaning_data = cleaning_data.withColumn("Device_Android_Version_Dependency", f.when(f.col("Android Ver").isNull(), True).otherwise(False))

In [15]:
cleaning_data.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+------------+---------------------------------+----------------------+---------------------------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|Last Updated|Current Ver| Android Ver|Device_Current_Version_Dependency|Device_Size_Dependency|Device_Android_Version_Dependency|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+------------+---------------------------------+----------------------+---------------------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|  2018-01-07|      1.0.0|4.0.3 and up|                            false|                 false|                            false|
| Coloring book 

In [16]:
cleaning_data_test = cleaning_data.withColumn(
    "Size_in_Megabytes",
    f.when(cleaning_data["Size"].contains("M"), 
         f.regexp_replace(cleaning_data["Size"], "M", "").cast("float"))
    .when(cleaning_data["Size"].contains("k"), 
          (f.regexp_replace(cleaning_data["Size"], "k", "").cast("float") / 1024))
    .otherwise(cleaning_data["Size"])
)


In [17]:
cleaning_data_test.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+------------+---------------------------------+----------------------+---------------------------------+-----------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|Last Updated|Current Ver| Android Ver|Device_Current_Version_Dependency|Device_Size_Dependency|Device_Android_Version_Dependency|Size_in_Megabytes|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+------------+---------------------------------+----------------------+---------------------------------+-----------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|  2018-01-07|      1.0.0|4.0.3 and up|                            false|                 fal

In [19]:
cleaning_data = cleaning_data_test.withColumn("Android Ver", f.regexp_replace(cleaning_data_test["Android Ver"], " and up", ""))

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+-----------+---------------------------------+----------------------+---------------------------------+-----------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|Last Updated|Current Ver|Android Ver|Device_Current_Version_Dependency|Device_Size_Dependency|Device_Android_Version_Dependency|Size_in_Megabytes|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+-----------+-----------+---------------------------------+----------------------+---------------------------------+-----------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|  2018-01-07|      1.0.0|      4.0.3|                            false|                 false| 

In [21]:

csv_path = "./Data/"
cleaning_data.write.csv(csv_path, header=True, mode = "overwrite")



In [22]:
parquet_path = "./Data/Parquet_Data/"
cleaning_data.write.parquet(parquet_path, mode="overwrite")

In [23]:
cleaning_data.write.format('jdbc').options(url=jdbc_url,driver = 'org.postgresql.Driver', dbtable = 'stagged_google_playstore_data', user='postgres',password='1234').mode('overwrite').save()